<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/aranza2/Training_Clustering_Test_Aranza3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install pyspark

     |████████████████████████████████| 204.7MB 41kB/s 
     |████████████████████████████████| 204kB 63.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=3221624258871137084353056321bd8dd3385d0c5c4e30a5f862ee2716b5a65f
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [1]:
pip install tensorflow

In [2]:
pip install --upgrade tensorflow-datasets

     |████████████████████████████████| 3.4MB 4.2MB/s 
  Found existing installation: tensorflow-datasets 2.1.0
    Uninstalling tensorflow-datasets-2.1.0:
      Successfully uninstalled tensorflow-datasets-2.1.0


In [3]:
pip install -q tensorflow>=2 tfds-nightly matplotlib

In [60]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds 
import pandas as pd

In [61]:
# Loading the wikipedia dataset.
DATASET_NAME = 'wikipedia/20190301.en'
# DATASET_NAME = 'wikipedia/20190301.uk'

dataset, dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    split=tfds.Split.TRAIN,
)

In [62]:
print(dataset_info)

tfds.core.DatasetInfo(
    name='wikipedia',
    version=1.0.0,
    description='Wikipedia dataset containing cleaned articles of all languages.
The datasets are built from the Wikipedia dump
(https://dumps.wikimedia.org/) with one split per language. Each example
contains the content of one full Wikipedia article with cleaning to strip
markdown and unwanted sections (references, etc.).',
    homepage='https://dumps.wikimedia.org',
    features=FeaturesDict({
        'text': Text(shape=(), dtype=tf.string),
        'title': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=5824596,
    splits={
        'train': 5824596,
    },
    supervised_keys=None,
    citation="""@ONLINE {wikidump,
        author = "Wikimedia Foundation",
        title  = "Wikimedia Downloads",
        url    = "https://dumps.wikimedia.org"
    }""",
    redistribution_info=license: "This work is licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License. To view a copy of this l

In [63]:
print(dataset)

<PrefetchDataset shapes: {text: (), title: ()}, types: {text: tf.string, title: tf.string}>


In [64]:
#Find how many train examples we have available
TRAIN_NUM_EXAMPLES = dataset_info.splits['train'].num_examples
print('Total number of articles: ', TRAIN_NUM_EXAMPLES)

Total number of articles:  5824596


In [65]:
#Print first article to view as an example
print('First article','\n======\n')
for example in dataset.take(2):
    print('Title:','\n------')
    print(example['title'].numpy().decode('utf-8'))
    print()

    print('Text:', '\n------')
    print(example['text'].numpy().decode('utf-8'))

First article 

Title: 
------
Joseph Greenberg

Text: 
------
Joseph Harold Greenberg (May 28, 1915 – May 7, 2001) was an American linguist, known mainly for his work concerning linguistic typology and the genetic classification of languages.

Life

Early life and education 
(Main source: Croft 2003)

Joseph Greenberg was born on May 28, 1915 to Jewish parents in Brooklyn, New York. His first great interest was music. At the age of 14, he gave a piano concert in Steinway Hall. He continued to play the piano frequently throughout his life.

After finishing high school, he decided to pursue a scholarly career rather than a musical one. He enrolled at Columbia University in New York. During his senior year, he attended a class taught by Franz Boas concerning American Indian languages. With references from Boas and Ruth Benedict, he was accepted as a graduate student by Melville J. Herskovits at Northwestern University in Chicago. During the course of his graduate studies, Greenberg did f

In [66]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
 # Start a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [125]:
word_rdd = spark.sparkContext.emptyRDD()
word_list = []
stop_words = set(stopwords.words("english"))
punctuations = [".", ",", "!", "?", ";", "#", ":", "(", ")", "-", "–" "/", "http", "https", "www", "\\", "@", "&", "[", "]","~"]

for article in dataset.take(2):
    sentences = nltk.sent_tokenize(article['text'].numpy().decode('utf-8'))
    # word_rdd = sentences.map(lambda line: list(map(lambda x:spark.sparkContext.parallelize(x.split(" ")), sentences)))
    for sentence in sentences:
      words = nltk.word_tokenize(sentence)
      without_stop_words = [
      word for word in words if word not in stop_words]
      clean_words = [word for word in without_stop_words if word not in punctuations]
      # word_rdd = word_rdd.union(spark.sparkContext.parallelize(clean_words))
      word_list.append(clean_words)
      par_words = spark.sparkContext.parallelize(clean_words)
      word_rdd = word_rdd.join(par_words)


In [128]:
word_rdd

PythonRDD[7527] at RDD at PythonRDD.scala:53

In [129]:
word_rdd.collect()

Py4JJavaError: ignored

In [79]:
#word_list = word_list.map(lambda x: x)

In [80]:
#type(word_list)

pyspark.rdd.PipelinedRDD

In [85]:
#word_list

PythonRDD[3339] at RDD at PythonRDD.scala:53

In [84]:
#word_list.collect()

['Joseph',
 'Harold',
 'Greenberg',
 'May',
 '28',
 '1915',
 '–',
 'May',
 '7',
 '2001',
 'American',
 'linguist',
 'known',
 'mainly',
 'work',
 'concerning',
 'linguistic',
 'typology',
 'genetic',
 'classification',
 'languages',
 'Life',
 'Early',
 'life',
 'education',
 'Main',
 'source',
 'Croft',
 '2003',
 'Joseph',
 'Greenberg',
 'born',
 'May',
 '28',
 '1915',
 'Jewish',
 'parents',
 'Brooklyn',
 'New',
 'York',
 'His',
 'first',
 'great',
 'interest',
 'music',
 'At',
 'age',
 '14',
 'gave',
 'piano',
 'concert',
 'Steinway',
 'Hall',
 'He',
 'continued',
 'play',
 'piano',
 'frequently',
 'throughout',
 'life',
 'After',
 'finishing',
 'high',
 'school',
 'decided',
 'pursue',
 'scholarly',
 'career',
 'rather',
 'musical',
 'one',
 'He',
 'enrolled',
 'Columbia',
 'University',
 'New',
 'York',
 'During',
 'senior',
 'year',
 'attended',
 'class',
 'taught',
 'Franz',
 'Boas',
 'concerning',
 'American',
 'Indian',
 'languages',
 'With',
 'references',
 'Boas',
 'Ruth',
 'B

In [94]:
# # Note this is an RDD approach, not dataframe. API is slightly different for dataframe
from pyspark.mllib.feature import Word2Vec
word2vec = Word2Vec()
model = word2vec.fit(par_words)

In [95]:
synonyms = model.findSynonyms('order', 1)

for word, cosine_distance in synonyms:
     print("{}: {}".format(word, cosine_distance))

Research: 0.9561575651168823


In [ ]:
def cosine_similarity(x, y):
     return x.dot(y)/(x.norm(2)*y.norm(2))

In [ ]:
x = model.transform('coming')
y = model.transform('bond')
cosine_similarity(x, y)

In [ ]:
# synonyms = model.findSynonyms('diner', 2)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

In [ ]:
# def cosine_similarity(x, y):
#     return x.dot(y)/(x.norm(2)*y.norm(2))

# x = model.transform('diner')
# y = model.transform('slice,')
# cosine_similarity(x, y)